In [ ]:
import struct
import os
import numpy as np
import pandas as pd
import random
import plotly.graph_objects as go
import math
import sys
import datetime

In [ ]:
PVALUES_MIXEDSPECTRACRUX="/mnt/f633ac7c-3153-4566-a009-229a0ae5f8a1/unicamp/doutorado/clustering/linfeng/sample/mixedSpectraCrux_experiments_pvalues/300.dat_allSpectraPvalues.bin"
COUNTS_MIXEDSPECTRACRUX="/media/eduseiti/ebee9cb2-c63c-48bf-b862-004aba7612ee/unicamp/mixedSpectraCrux/mixedSpectraCrux_experiment_spectra_count.tsv"

# PVALUES_FILE="/mnt/f633ac7c-3153-4566-a009-229a0ae5f8a1/unicamp/doutorado/clustering/linfeng/sample/test_pvalues_fix/400.dat_allSpectraPvalues.bin"
# IDS_FILE="/mnt/f633ac7c-3153-4566-a009-229a0ae5f8a1/unicamp/doutorado/clustering/linfeng/sample/identifications_sample_0.1_nterm/sample_experiment_identifications/percolator.target.psms.txt"
# COUNTS_FILE="/mnt/f633ac7c-3153-4566-a009-229a0ae5f8a1/unicamp/doutorado/spectra_count.tsv"

In [ ]:
ADULT_ADRENALGLAND_GEL_ELITE_CRUX_FILES = [
    'Adult_Adrenalgland_Gel_Elite_49_f01.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f02.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f03.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f04.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f05.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f06.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f07.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f08.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f09.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f10.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f11.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f12.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f13.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f14.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f15.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f16.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f17.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f18.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f19.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f20.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f21.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f22.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f23.mgf',
    'Adult_Adrenalgland_Gel_Elite_49_f24.mgf'
]

ADULT_ADRENALGLAND_GEL_VELOS_CRUX_FILES = [
    'Adult_Adrenalgland_Gel_Velos_2_f01.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f02.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f03.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f04.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f05.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f06.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f07.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f08.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f09.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f10.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f11.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f12.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f13.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f14.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f15.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f16.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f17.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f18.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f19.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f20.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f21.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f22.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f23.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f24.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f25.mgf',
    'Adult_Adrenalgland_Gel_Velos_2_f26.mgf'
]

ADULT_ADRENALGLAND_BRP_VELOS_CRUX_FILES = [
    'Adult_Adrenalgland_bRP_Velos_1_f01.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f02.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f03.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f04.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f05.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f06.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f07.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f08.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f09.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f10.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f11.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f12.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f13.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f14.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f15.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f16.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f17.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f18.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f19.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f20.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f21.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f22.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f23.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f24.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f25.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f26.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f27.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f28.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f29.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f30.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f31.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f32.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f33.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f34.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f35.mgf',
    'Adult_Adrenalgland_bRP_Velos_1_f36.mgf'
]

ADULT_HEART_BRP_ELITE_CRUX_FILES = [
    'Adult_Heart_bRP_Elite_81_f01.mgf',
    'Adult_Heart_bRP_Elite_81_f02.mgf',
    'Adult_Heart_bRP_Elite_81_f03.mgf',
    'Adult_Heart_bRP_Elite_81_f04.mgf',
    'Adult_Heart_bRP_Elite_81_f05.mgf',
    'Adult_Heart_bRP_Elite_81_f06.mgf',
    'Adult_Heart_bRP_Elite_81_f07.mgf',
    'Adult_Heart_bRP_Elite_81_f08.mgf',
    'Adult_Heart_bRP_Elite_81_f09.mgf',
    'Adult_Heart_bRP_Elite_81_f10.mgf',
    'Adult_Heart_bRP_Elite_81_f11.mgf',
    'Adult_Heart_bRP_Elite_81_f12.mgf',
    'Adult_Heart_bRP_Elite_81_f13.mgf',
    'Adult_Heart_bRP_Elite_81_f14.mgf',
    'Adult_Heart_bRP_Elite_81_f15.mgf',
    'Adult_Heart_bRP_Elite_81_f16.mgf',
    'Adult_Heart_bRP_Elite_81_f17.mgf',
    'Adult_Heart_bRP_Elite_81_f18.mgf',
    'Adult_Heart_bRP_Elite_81_f19.mgf',
    'Adult_Heart_bRP_Elite_81_f20.mgf',
    'Adult_Heart_bRP_Elite_81_f21.mgf',
    'Adult_Heart_bRP_Elite_81_f22.mgf',
    'Adult_Heart_bRP_Elite_81_f23.mgf',
    'Adult_Heart_bRP_Elite_81_f24.mgf'
]

ADULT_HEART_BRP_VELOS_CRUX_FILES = [
    'Adult_Heart_bRP_Velos_6_f01.mgf',
    'Adult_Heart_bRP_Velos_6_f02.mgf',
    'Adult_Heart_bRP_Velos_6_f03.mgf',
    'Adult_Heart_bRP_Velos_6_f04.mgf',
    'Adult_Heart_bRP_Velos_6_f05.mgf',
    'Adult_Heart_bRP_Velos_6_f06.mgf',
    'Adult_Heart_bRP_Velos_6_f07.mgf',
    'Adult_Heart_bRP_Velos_6_f08.mgf',
    'Adult_Heart_bRP_Velos_6_f09.mgf',
    'Adult_Heart_bRP_Velos_6_f10.mgf',
    'Adult_Heart_bRP_Velos_6_f11.mgf',
    'Adult_Heart_bRP_Velos_6_f12.mgf',
    'Adult_Heart_bRP_Velos_6_f13.mgf',
    'Adult_Heart_bRP_Velos_6_f14.mgf',
    'Adult_Heart_bRP_Velos_6_f15.mgf',
    'Adult_Heart_bRP_Velos_6_f16.mgf',
    'Adult_Heart_bRP_Velos_6_f17.mgf',
    'Adult_Heart_bRP_Velos_6_f18.mgf',
    'Adult_Heart_bRP_Velos_6_f19.mgf',
    'Adult_Heart_bRP_Velos_6_f20.mgf',
    'Adult_Heart_bRP_Velos_6_f21.mgf',
    'Adult_Heart_bRP_Velos_6_f22.mgf',
    'Adult_Heart_bRP_Velos_6_f23.mgf',
    'Adult_Heart_bRP_Velos_6_f24.mgf',
    'Adult_Heart_bRP_Velos_6_f25.mgf',
    'Adult_Heart_bRP_Velos_6_f26.mgf'
]

ADULT_PLATELETS_GEL_ELITE_CRUX_FILES = [
    'Adult_Platelets_Gel_Elite_48_f01.mgf',
    'Adult_Platelets_Gel_Elite_48_f02.mgf',
    'Adult_Platelets_Gel_Elite_48_f03.mgf',
    'Adult_Platelets_Gel_Elite_48_f04.mgf',
    'Adult_Platelets_Gel_Elite_48_f05.mgf',
    'Adult_Platelets_Gel_Elite_48_f06.mgf',
    'Adult_Platelets_Gel_Elite_48_f07.mgf',
    'Adult_Platelets_Gel_Elite_48_f08.mgf',
    'Adult_Platelets_Gel_Elite_48_f09.mgf',
    'Adult_Platelets_Gel_Elite_48_f10.mgf',
    'Adult_Platelets_Gel_Elite_48_f11.mgf',
    'Adult_Platelets_Gel_Elite_48_f12.mgf',
    'Adult_Platelets_Gel_Elite_48_f13.mgf',
    'Adult_Platelets_Gel_Elite_48_f14.mgf',
    'Adult_Platelets_Gel_Elite_48_f15.mgf',
    'Adult_Platelets_Gel_Elite_48_f16.mgf',
    'Adult_Platelets_Gel_Elite_48_f17.mgf',
    'Adult_Platelets_Gel_Elite_48_f18.mgf',
    'Adult_Platelets_Gel_Elite_48_f19.mgf',
    'Adult_Platelets_Gel_Elite_48_f20.mgf',
    'Adult_Platelets_Gel_Elite_48_f21.mgf',
    'Adult_Platelets_Gel_Elite_48_f22.mgf',
    'Adult_Platelets_Gel_Elite_48_f23.mgf',
    'Adult_Platelets_Gel_Elite_48_f24.mgf'
]

ADULT_URINARYBLADDER_GEL_ELITE_CRUX_FILES = [
    'Adult_Urinarybladder_Gel_Elite_70_f01.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f02.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f03.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f04.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f05.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f06.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f07.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f08.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f09.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f10.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f11.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f12.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f13.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f14.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f15.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f16.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f17.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f18.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f19.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f20.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f21.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f22.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f23.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f24.mgf',
    'Adult_Urinarybladder_Gel_Elite_70_f25.mgf'
]

FETAL_BRAIN_GEL_VELOS_CRUX_FILES = [
    "Fetal_Brain_Gel_Velos_16_f01.mgf",
    "Fetal_Brain_Gel_Velos_16_f02.mgf",
    "Fetal_Brain_Gel_Velos_16_f03.mgf",
    "Fetal_Brain_Gel_Velos_16_f04.mgf",
    "Fetal_Brain_Gel_Velos_16_f05.mgf",
    "Fetal_Brain_Gel_Velos_16_f06.mgf",
    "Fetal_Brain_Gel_Velos_16_f07.mgf",
    "Fetal_Brain_Gel_Velos_16_f08.mgf",
    "Fetal_Brain_Gel_Velos_16_f09.mgf",
    "Fetal_Brain_Gel_Velos_16_f10.mgf",
    "Fetal_Brain_Gel_Velos_16_f11.mgf",
    "Fetal_Brain_Gel_Velos_16_f12.mgf",
    "Fetal_Brain_Gel_Velos_16_f13.mgf",
    "Fetal_Brain_Gel_Velos_16_f14.mgf",
    "Fetal_Brain_Gel_Velos_16_f15.mgf",
    "Fetal_Brain_Gel_Velos_16_f16.mgf",
    "Fetal_Brain_Gel_Velos_16_f17.mgf",
    "Fetal_Brain_Gel_Velos_16_f18.mgf",
    "Fetal_Brain_Gel_Velos_16_f19.mgf",
    "Fetal_Brain_Gel_Velos_16_f20.mgf",
    "Fetal_Brain_Gel_Velos_16_f21.mgf",
    "Fetal_Brain_Gel_Velos_16_f22.mgf",
    "Fetal_Brain_Gel_Velos_16_f23.mgf",
    "Fetal_Brain_Gel_Velos_16_f24.mgf",
    "Fetal_Brain_Gel_Velos_16_f25.mgf",
    "Fetal_Brain_Gel_Velos_16_f26.mgf",
    "Fetal_Brain_Gel_Velos_16_f27.mgf",
    "Fetal_Brain_Gel_Velos_16_f28.mgf",
    "Fetal_Brain_Gel_Velos_16_f29.mgf"
]

FETAL_LIVER_BRP_ELITE_CRUX_FILES = [
    'Fetal_Liver_bRP_Elite_22_f01.mgf',
    'Fetal_Liver_bRP_Elite_22_f02.mgf',
    'Fetal_Liver_bRP_Elite_22_f03.mgf',
    'Fetal_Liver_bRP_Elite_22_f04.mgf',
    'Fetal_Liver_bRP_Elite_22_f05.mgf',
    'Fetal_Liver_bRP_Elite_22_f06.mgf',
    'Fetal_Liver_bRP_Elite_22_f07.mgf',
    'Fetal_Liver_bRP_Elite_22_f08.mgf',
    'Fetal_Liver_bRP_Elite_22_f09.mgf',
    'Fetal_Liver_bRP_Elite_22_f10.mgf',
    'Fetal_Liver_bRP_Elite_22_f11.mgf',
    'Fetal_Liver_bRP_Elite_22_f12.mgf',
    'Fetal_Liver_bRP_Elite_22_f13.mgf',
    'Fetal_Liver_bRP_Elite_22_f14.mgf',
    'Fetal_Liver_bRP_Elite_22_f15.mgf',
    'Fetal_Liver_bRP_Elite_22_f16.mgf',
    'Fetal_Liver_bRP_Elite_22_f17.mgf',
    'Fetal_Liver_bRP_Elite_22_f18.mgf',
    'Fetal_Liver_bRP_Elite_22_f19.mgf',
    'Fetal_Liver_bRP_Elite_22_f20.mgf',
    'Fetal_Liver_bRP_Elite_22_f21.mgf',
    'Fetal_Liver_bRP_Elite_22_f22.mgf',
    'Fetal_Liver_bRP_Elite_22_f23.mgf',
    'Fetal_Liver_bRP_Elite_22_f24.mgf',
    'Fetal_Liver_bRP_Elite_22_f25.mgf',
    'Fetal_Liver_bRP_Elite_22_f26.mgf',
    'Fetal_Liver_bRP_Elite_22_f27.mgf',
    'Fetal_Liver_bRP_Elite_22_f28.mgf',
    'Fetal_Liver_bRP_Elite_22_f29.mgf',
    'Fetal_Liver_bRP_Elite_22_f30.mgf',
    'Fetal_Liver_bRP_Elite_22_f31.mgf',
    'Fetal_Liver_bRP_Elite_22_f32.mgf',
    'Fetal_Liver_bRP_Elite_22_f33.mgf',
    'Fetal_Liver_bRP_Elite_22_f34.mgf',
    'Fetal_Liver_bRP_Elite_22_f35.mgf'
]

FETAL_OVARY_BRP_VELOS_CRUX_FILES = [
    'Fetal_Ovary_bRP_Velos_41_f01.mgf',
    'Fetal_Ovary_bRP_Velos_41_f02.mgf',
    'Fetal_Ovary_bRP_Velos_41_f03.mgf',
    'Fetal_Ovary_bRP_Velos_41_f04.mgf',
    'Fetal_Ovary_bRP_Velos_41_f05.mgf',
    'Fetal_Ovary_bRP_Velos_41_f06.mgf',
    'Fetal_Ovary_bRP_Velos_41_f07.mgf',
    'Fetal_Ovary_bRP_Velos_41_f08.mgf',
    'Fetal_Ovary_bRP_Velos_41_f09.mgf',
    'Fetal_Ovary_bRP_Velos_41_f10.mgf',
    'Fetal_Ovary_bRP_Velos_41_f11.mgf',
    'Fetal_Ovary_bRP_Velos_41_f12.mgf',
    'Fetal_Ovary_bRP_Velos_41_f13.mgf',
    'Fetal_Ovary_bRP_Velos_41_f14.mgf',
    'Fetal_Ovary_bRP_Velos_41_f15.mgf',
    'Fetal_Ovary_bRP_Velos_41_f16.mgf',
    'Fetal_Ovary_bRP_Velos_41_f17.mgf',
    'Fetal_Ovary_bRP_Velos_41_f18.mgf',
    'Fetal_Ovary_bRP_Velos_41_f19.mgf',
    'Fetal_Ovary_bRP_Velos_41_f20.mgf',
    'Fetal_Ovary_bRP_Velos_41_f21.mgf',
    'Fetal_Ovary_bRP_Velos_41_f22.mgf',
    'Fetal_Ovary_bRP_Velos_41_f23.mgf',
    'Fetal_Ovary_bRP_Velos_41_f24.mgf',
    'Fetal_Ovary_bRP_Velos_41_f25.mgf',
    'Fetal_Ovary_bRP_Velos_41_f26.mgf'
]

FETAL_OVARY_BRP_ELITE_CRUX_FILES = [
    'Fetal_Ovary_bRP_Elite_25_f01.mgf',
    'Fetal_Ovary_bRP_Elite_25_f02.mgf',
    'Fetal_Ovary_bRP_Elite_25_f03.mgf',
    'Fetal_Ovary_bRP_Elite_25_f04.mgf',
    'Fetal_Ovary_bRP_Elite_25_f05.mgf',
    'Fetal_Ovary_bRP_Elite_25_f06.mgf',
    'Fetal_Ovary_bRP_Elite_25_f07.mgf',
    'Fetal_Ovary_bRP_Elite_25_f08.mgf',
    'Fetal_Ovary_bRP_Elite_25_f09.mgf',
    'Fetal_Ovary_bRP_Elite_25_f10.mgf',
    'Fetal_Ovary_bRP_Elite_25_f11.mgf',
    'Fetal_Ovary_bRP_Elite_25_f12.mgf',
    'Fetal_Ovary_bRP_Elite_25_f13.mgf',
    'Fetal_Ovary_bRP_Elite_25_f14.mgf',
    'Fetal_Ovary_bRP_Elite_25_f15.mgf',
    'Fetal_Ovary_bRP_Elite_25_f16.mgf',
    'Fetal_Ovary_bRP_Elite_25_f17.mgf',
    'Fetal_Ovary_bRP_Elite_25_f18.mgf',
    'Fetal_Ovary_bRP_Elite_25_f19.mgf',
    'Fetal_Ovary_bRP_Elite_25_f20.mgf',
    'Fetal_Ovary_bRP_Elite_25_f21.mgf',
    'Fetal_Ovary_bRP_Elite_25_f22.mgf',
    'Fetal_Ovary_bRP_Elite_25_f23.mgf',
    'Fetal_Ovary_bRP_Elite_25_f24.mgf'
]



ADULT_BCELLS_BRP_ELITE_CRUX_FILES = [
    "Adult_Bcells_bRP_Elite_75_f01.mgf",
    "Adult_Bcells_bRP_Elite_75_f02.mgf",
    "Adult_Bcells_bRP_Elite_75_f03.mgf",
    "Adult_Bcells_bRP_Elite_75_f04.mgf",
    "Adult_Bcells_bRP_Elite_75_f05.mgf",
    "Adult_Bcells_bRP_Elite_75_f06.mgf",
    "Adult_Bcells_bRP_Elite_75_f07.mgf",
    "Adult_Bcells_bRP_Elite_75_f08.mgf",
    "Adult_Bcells_bRP_Elite_75_f09.mgf",
    "Adult_Bcells_bRP_Elite_75_f10.mgf",
    "Adult_Bcells_bRP_Elite_75_f11.mgf",
    "Adult_Bcells_bRP_Elite_75_f12.mgf",
    "Adult_Bcells_bRP_Elite_75_f13.mgf",
    "Adult_Bcells_bRP_Elite_75_f14.mgf",
    "Adult_Bcells_bRP_Elite_75_f15.mgf",
    "Adult_Bcells_bRP_Elite_75_f16.mgf",
    "Adult_Bcells_bRP_Elite_75_f17.mgf",
    "Adult_Bcells_bRP_Elite_75_f18.mgf",
    "Adult_Bcells_bRP_Elite_75_f19.mgf",
    "Adult_Bcells_bRP_Elite_75_f20.mgf",
    "Adult_Bcells_bRP_Elite_75_f21.mgf",
    "Adult_Bcells_bRP_Elite_75_f22.mgf",
    "Adult_Bcells_bRP_Elite_75_f23.mgf",
    "Adult_Bcells_bRP_Elite_75_f24.mgf"
]

ADULT_BCELLS_BRP_VELOS_CRUX_FILES = [
    "Adult_Bcells_bRP_Velos_42_f01.mgf",
    "Adult_Bcells_bRP_Velos_42_f02.mgf",
    "Adult_Bcells_bRP_Velos_42_f03.mgf",
    "Adult_Bcells_bRP_Velos_42_f04.mgf",
    "Adult_Bcells_bRP_Velos_42_f05.mgf",
    "Adult_Bcells_bRP_Velos_42_f06.mgf",
    "Adult_Bcells_bRP_Velos_42_f07.mgf",
    "Adult_Bcells_bRP_Velos_42_f08.mgf",
    "Adult_Bcells_bRP_Velos_42_f09.mgf",
    "Adult_Bcells_bRP_Velos_42_f10.mgf",
    "Adult_Bcells_bRP_Velos_42_f11.mgf",
    "Adult_Bcells_bRP_Velos_42_f12.mgf",
    "Adult_Bcells_bRP_Velos_42_f13.mgf",
    "Adult_Bcells_bRP_Velos_42_f14.mgf",
    "Adult_Bcells_bRP_Velos_42_f15.mgf",
    "Adult_Bcells_bRP_Velos_42_f16.mgf",
    "Adult_Bcells_bRP_Velos_42_f17.mgf",
    "Adult_Bcells_bRP_Velos_42_f18.mgf",
    "Adult_Bcells_bRP_Velos_42_f19.mgf",
    "Adult_Bcells_bRP_Velos_42_f20.mgf",
    "Adult_Bcells_bRP_Velos_42_f21.mgf",
    "Adult_Bcells_bRP_Velos_42_f22.mgf",
    "Adult_Bcells_bRP_Velos_42_f23.mgf",
    "Adult_Bcells_bRP_Velos_42_f24.mgf",
    "Adult_Bcells_bRP_Velos_42_f25.mgf",
    "Adult_Bcells_bRP_Velos_42_f26.mgf",
    "Adult_Bcells_bRP_Velos_42_f27.mgf",
    "Adult_Bcells_bRP_Velos_42_f28.mgf",
    "Adult_Bcells_bRP_Velos_42_f29.mgf",
    "Adult_Bcells_bRP_Velos_42_f30.mgf",
    "Adult_Bcells_bRP_Velos_42_f31.mgf",
    "Adult_Bcells_bRP_Velos_42_f32.mgf"
]

ADULT_BCELLS_GEL_ELITE_CRUX_FILES = [
    "Adult_Bcells_Gel_Elite_76_f01.mgf",
    "Adult_Bcells_Gel_Elite_76_f02.mgf",
    "Adult_Bcells_Gel_Elite_76_f03.mgf",
    "Adult_Bcells_Gel_Elite_76_f04.mgf",
    "Adult_Bcells_Gel_Elite_76_f05.mgf",
    "Adult_Bcells_Gel_Elite_76_f06.mgf",
    "Adult_Bcells_Gel_Elite_76_f07.mgf",
    "Adult_Bcells_Gel_Elite_76_f08.mgf",
    "Adult_Bcells_Gel_Elite_76_f09.mgf",
    "Adult_Bcells_Gel_Elite_76_f10.mgf",
    "Adult_Bcells_Gel_Elite_76_f11.mgf",
    "Adult_Bcells_Gel_Elite_76_f12.mgf",
    "Adult_Bcells_Gel_Elite_76_f13.mgf",
    "Adult_Bcells_Gel_Elite_76_f14.mgf",
    "Adult_Bcells_Gel_Elite_76_f15.mgf",
    "Adult_Bcells_Gel_Elite_76_f16.mgf",
    "Adult_Bcells_Gel_Elite_76_f17.mgf",
    "Adult_Bcells_Gel_Elite_76_f18.mgf",
    "Adult_Bcells_Gel_Elite_76_f19.mgf",
    "Adult_Bcells_Gel_Elite_76_f20.mgf",
    "Adult_Bcells_Gel_Elite_76_f21.mgf",
    "Adult_Bcells_Gel_Elite_76_f22.mgf",
    "Adult_Bcells_Gel_Elite_76_f23.mgf",
    "Adult_Bcells_Gel_Elite_76_f24.mgf"
]

ADULT_CD4TCELLS_GEL_VELOS_CRUX_FILES = [
    "Adult_CD4Tcells_Gel_Velos_30_f01.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f02.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f03.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f04.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f05.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f06.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f07.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f08.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f09.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f10.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f11.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f12.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f13.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f14.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f15.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f16.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f17.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f18.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f19.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f20.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f21.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f22.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f23.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f24.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f25.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f26.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f27.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f28.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f29.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f30.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f31.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f32.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f33.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f34.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f35.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f36.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f37.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f38.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f39.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f40.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f41.mgf",
    "Adult_CD4Tcells_Gel_Velos_30_f42.mgf"
]

ADULT_CD8TCELLS_GEL_ELITE_CRUX_FILES = [
    "Adult_CD8Tcells_Gel_Elite_44_f01.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f02.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f03.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f04.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f05.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f06.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f07.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f08.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f09.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f10.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f11.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f12.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f13.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f14.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f15.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f16.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f17.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f18.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f19.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f20.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f21.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f22.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f23.mgf",
    "Adult_CD8Tcells_Gel_Elite_44_f24.mgf"
]

ADULT_COLON_GEL_ELITE_CRUX_FILES = [
    "Adult_Colon_Gel_Elite_51_f01.mgf",
    "Adult_Colon_Gel_Elite_51_f02.mgf",
    "Adult_Colon_Gel_Elite_51_f03.mgf",
    "Adult_Colon_Gel_Elite_51_f04.mgf",
    "Adult_Colon_Gel_Elite_51_f05.mgf",
    "Adult_Colon_Gel_Elite_51_f06.mgf",
    "Adult_Colon_Gel_Elite_51_f07.mgf",
    "Adult_Colon_Gel_Elite_51_f08.mgf",
    "Adult_Colon_Gel_Elite_51_f09.mgf",
    "Adult_Colon_Gel_Elite_51_f10.mgf",
    "Adult_Colon_Gel_Elite_51_f11.mgf",
    "Adult_Colon_Gel_Elite_51_f12.mgf",
    "Adult_Colon_Gel_Elite_51_f13.mgf",
    "Adult_Colon_Gel_Elite_51_f14.mgf",
    "Adult_Colon_Gel_Elite_51_f15.mgf",
    "Adult_Colon_Gel_Elite_51_f16.mgf",
    "Adult_Colon_Gel_Elite_51_f17.mgf",
    "Adult_Colon_Gel_Elite_51_f18.mgf",
    "Adult_Colon_Gel_Elite_51_f19.mgf",
    "Adult_Colon_Gel_Elite_51_f20.mgf",
    "Adult_Colon_Gel_Elite_51_f21.mgf",
    "Adult_Colon_Gel_Elite_51_f22.mgf",
    "Adult_Colon_Gel_Elite_51_f23.mgf",
    "Adult_Colon_Gel_Elite_51_f24.mgf"
]

ADULT_ESOPHAGUS_GEL_VELOS_CRUX_FILES = [
    "Adult_Esophagus_Gel_Velos_4_f01.mgf",
    "Adult_Esophagus_Gel_Velos_4_f02.mgf",
    "Adult_Esophagus_Gel_Velos_4_f03.mgf",
    "Adult_Esophagus_Gel_Velos_4_f04.mgf",
    "Adult_Esophagus_Gel_Velos_4_f05.mgf",
    "Adult_Esophagus_Gel_Velos_4_f06.mgf",
    "Adult_Esophagus_Gel_Velos_4_f07.mgf",
    "Adult_Esophagus_Gel_Velos_4_f08.mgf",
    "Adult_Esophagus_Gel_Velos_4_f09.mgf",
    "Adult_Esophagus_Gel_Velos_4_f10.mgf",
    "Adult_Esophagus_Gel_Velos_4_f11.mgf",
    "Adult_Esophagus_Gel_Velos_4_f12.mgf",
    "Adult_Esophagus_Gel_Velos_4_f13.mgf",
    "Adult_Esophagus_Gel_Velos_4_f14.mgf",
    "Adult_Esophagus_Gel_Velos_4_f15.mgf",
    "Adult_Esophagus_Gel_Velos_4_f16.mgf",
    "Adult_Esophagus_Gel_Velos_4_f17.mgf",
    "Adult_Esophagus_Gel_Velos_4_f18.mgf",
    "Adult_Esophagus_Gel_Velos_4_f19.mgf",
    "Adult_Esophagus_Gel_Velos_4_f20.mgf",
    "Adult_Esophagus_Gel_Velos_4_f21.mgf",
    "Adult_Esophagus_Gel_Velos_4_f22.mgf",
    "Adult_Esophagus_Gel_Velos_4_f23.mgf",
    "Adult_Esophagus_Gel_Velos_4_f24.mgf"
]

ADULT_NKCELLS_BRP_ELITE_CRUX_FILES = [
    "Adult_NKcells_bRP_Elite_34_f01.mgf",
    "Adult_NKcells_bRP_Elite_34_f02.mgf",
    "Adult_NKcells_bRP_Elite_34_f03.mgf",
    "Adult_NKcells_bRP_Elite_34_f04.mgf",
    "Adult_NKcells_bRP_Elite_34_f05.mgf",
    "Adult_NKcells_bRP_Elite_34_f06.mgf",
    "Adult_NKcells_bRP_Elite_34_f07.mgf",
    "Adult_NKcells_bRP_Elite_34_f08.mgf",
    "Adult_NKcells_bRP_Elite_34_f09.mgf",
    "Adult_NKcells_bRP_Elite_34_f10.mgf",
    "Adult_NKcells_bRP_Elite_34_f11.mgf",
    "Adult_NKcells_bRP_Elite_34_f12.mgf",
    "Adult_NKcells_bRP_Elite_34_f13.mgf",
    "Adult_NKcells_bRP_Elite_34_f14.mgf",
    "Adult_NKcells_bRP_Elite_34_f15.mgf",
    "Adult_NKcells_bRP_Elite_34_f16.mgf",
    "Adult_NKcells_bRP_Elite_34_f17.mgf",
    "Adult_NKcells_bRP_Elite_34_f18.mgf",
    "Adult_NKcells_bRP_Elite_34_f19.mgf",
    "Adult_NKcells_bRP_Elite_34_f20.mgf",
    "Adult_NKcells_bRP_Elite_34_f21.mgf",
    "Adult_NKcells_bRP_Elite_34_f22.mgf",
    "Adult_NKcells_bRP_Elite_34_f23.mgf",
    "Adult_NKcells_bRP_Elite_34_f24.mgf",
    "Adult_NKcells_bRP_Elite_34_f25.mgf",
    "Adult_NKcells_bRP_Elite_34_f26.mgf",
    "Adult_NKcells_bRP_Elite_34_f27.mgf",
    "Adult_NKcells_bRP_Elite_34_f28.mgf"
]

ADULT_NKCELLS_BRP_VELOS_CRUX_FILES = [
    "Adult_NKcells_bRP_Velos_46_f01.mgf",
    "Adult_NKcells_bRP_Velos_46_f02.mgf",
    "Adult_NKcells_bRP_Velos_46_f03.mgf",
    "Adult_NKcells_bRP_Velos_46_f04.mgf",
    "Adult_NKcells_bRP_Velos_46_f05.mgf",
    "Adult_NKcells_bRP_Velos_46_f06.mgf",
    "Adult_NKcells_bRP_Velos_46_f07.mgf",
    "Adult_NKcells_bRP_Velos_46_f08.mgf",
    "Adult_NKcells_bRP_Velos_46_f09.mgf",
    "Adult_NKcells_bRP_Velos_46_f10.mgf",
    "Adult_NKcells_bRP_Velos_46_f11.mgf",
    "Adult_NKcells_bRP_Velos_46_f12.mgf",
    "Adult_NKcells_bRP_Velos_46_f13.mgf",
    "Adult_NKcells_bRP_Velos_46_f14.mgf",
    "Adult_NKcells_bRP_Velos_46_f15.mgf",
    "Adult_NKcells_bRP_Velos_46_f16.mgf",
    "Adult_NKcells_bRP_Velos_46_f17.mgf",
    "Adult_NKcells_bRP_Velos_46_f18.mgf",
    "Adult_NKcells_bRP_Velos_46_f19.mgf",
    "Adult_NKcells_bRP_Velos_46_f20.mgf",
    "Adult_NKcells_bRP_Velos_46_f21.mgf",
    "Adult_NKcells_bRP_Velos_46_f22.mgf",
    "Adult_NKcells_bRP_Velos_46_f23.mgf",
    "Adult_NKcells_bRP_Velos_46_f24.mgf"
]

ADULT_PANCREAS_BRP_ELITE_CRUX_FILES = [
    "Adult_Pancreas_bRP_Elite_59_f01.mgf",
    "Adult_Pancreas_bRP_Elite_59_f02.mgf",
    "Adult_Pancreas_bRP_Elite_59_f03.mgf",
    "Adult_Pancreas_bRP_Elite_59_f04.mgf",
    "Adult_Pancreas_bRP_Elite_59_f05.mgf",
    "Adult_Pancreas_bRP_Elite_59_f06.mgf",
    "Adult_Pancreas_bRP_Elite_59_f07.mgf",
    "Adult_Pancreas_bRP_Elite_59_f08.mgf",
    "Adult_Pancreas_bRP_Elite_59_f09.mgf",
    "Adult_Pancreas_bRP_Elite_59_f10.mgf",
    "Adult_Pancreas_bRP_Elite_59_f11.mgf",
    "Adult_Pancreas_bRP_Elite_59_f12.mgf",
    "Adult_Pancreas_bRP_Elite_59_f13.mgf",
    "Adult_Pancreas_bRP_Elite_59_f14.mgf",
    "Adult_Pancreas_bRP_Elite_59_f15.mgf",
    "Adult_Pancreas_bRP_Elite_59_f16.mgf",
    "Adult_Pancreas_bRP_Elite_59_f17.mgf",
    "Adult_Pancreas_bRP_Elite_59_f18.mgf",
    "Adult_Pancreas_bRP_Elite_59_f19.mgf",
    "Adult_Pancreas_bRP_Elite_59_f20.mgf",
    "Adult_Pancreas_bRP_Elite_59_f21.mgf",
    "Adult_Pancreas_bRP_Elite_59_f22.mgf",
    "Adult_Pancreas_bRP_Elite_59_f23.mgf",
    "Adult_Pancreas_bRP_Elite_59_f24.mgf"
]

FETAL_GUT_GEL_VELOS_CRUX_FILES = [
    "Fetal_Gut_Gel_Velos_72_f01.mgf",
    "Fetal_Gut_Gel_Velos_72_f02.mgf",
    "Fetal_Gut_Gel_Velos_72_f03.mgf",
    "Fetal_Gut_Gel_Velos_72_f04.mgf",
    "Fetal_Gut_Gel_Velos_72_f05.mgf",
    "Fetal_Gut_Gel_Velos_72_f06.mgf",
    "Fetal_Gut_Gel_Velos_72_f07.mgf",
    "Fetal_Gut_Gel_Velos_72_f08.mgf",
    "Fetal_Gut_Gel_Velos_72_f09.mgf",
    "Fetal_Gut_Gel_Velos_72_f10.mgf",
    "Fetal_Gut_Gel_Velos_72_f11.mgf",
    "Fetal_Gut_Gel_Velos_72_f12.mgf",
    "Fetal_Gut_Gel_Velos_72_f13.mgf",
    "Fetal_Gut_Gel_Velos_72_f14.mgf",
    "Fetal_Gut_Gel_Velos_72_f15.mgf",
    "Fetal_Gut_Gel_Velos_72_f16.mgf",
    "Fetal_Gut_Gel_Velos_72_f17.mgf",
    "Fetal_Gut_Gel_Velos_72_f18.mgf",
    "Fetal_Gut_Gel_Velos_72_f19.mgf",
    "Fetal_Gut_Gel_Velos_72_f20.mgf",
    "Fetal_Gut_Gel_Velos_72_f21.mgf",
    "Fetal_Gut_Gel_Velos_72_f22.mgf",
    "Fetal_Gut_Gel_Velos_72_f23.mgf",
    "Fetal_Gut_Gel_Velos_72_f24.mgf"
]

FETAL_LIVER_BRP_ELITE_23_CRUX_FILES = [
    "Fetal_Liver_bRP_Elite_23_f01.mgf",
    "Fetal_Liver_bRP_Elite_23_f02.mgf",
    "Fetal_Liver_bRP_Elite_23_f03.mgf",
    "Fetal_Liver_bRP_Elite_23_f04.mgf",
    "Fetal_Liver_bRP_Elite_23_f05.mgf",
    "Fetal_Liver_bRP_Elite_23_f06.mgf",
    "Fetal_Liver_bRP_Elite_23_f07.mgf",
    "Fetal_Liver_bRP_Elite_23_f08.mgf",
    "Fetal_Liver_bRP_Elite_23_f09.mgf",
    "Fetal_Liver_bRP_Elite_23_f10.mgf",
    "Fetal_Liver_bRP_Elite_23_f11.mgf",
    "Fetal_Liver_bRP_Elite_23_f12.mgf",
    "Fetal_Liver_bRP_Elite_23_f13.mgf",
    "Fetal_Liver_bRP_Elite_23_f14.mgf",
    "Fetal_Liver_bRP_Elite_23_f15.mgf",
    "Fetal_Liver_bRP_Elite_23_f16.mgf",
    "Fetal_Liver_bRP_Elite_23_f17.mgf",
    "Fetal_Liver_bRP_Elite_23_f18.mgf",
    "Fetal_Liver_bRP_Elite_23_f19.mgf",
    "Fetal_Liver_bRP_Elite_23_f20.mgf",
    "Fetal_Liver_bRP_Elite_23_f21.mgf",
    "Fetal_Liver_bRP_Elite_23_f22.mgf",
    "Fetal_Liver_bRP_Elite_23_f23.mgf",
    "Fetal_Liver_bRP_Elite_23_f24.mgf"
]

FETAL_LIVER_GEL_VELOS_CRUX_FILES = [
    "Fetal_Liver_Gel_Velos_24_f01.mgf",
    "Fetal_Liver_Gel_Velos_24_f02.mgf",
    "Fetal_Liver_Gel_Velos_24_f03.mgf",
    "Fetal_Liver_Gel_Velos_24_f04.mgf",
    "Fetal_Liver_Gel_Velos_24_f05.mgf",
    "Fetal_Liver_Gel_Velos_24_f06.mgf",
    "Fetal_Liver_Gel_Velos_24_f07.mgf",
    "Fetal_Liver_Gel_Velos_24_f08.mgf",
    "Fetal_Liver_Gel_Velos_24_f09.mgf",
    "Fetal_Liver_Gel_Velos_24_f10.mgf",
    "Fetal_Liver_Gel_Velos_24_f11.mgf",
    "Fetal_Liver_Gel_Velos_24_f12.mgf",
    "Fetal_Liver_Gel_Velos_24_f13.mgf",
    "Fetal_Liver_Gel_Velos_24_f14.mgf",
    "Fetal_Liver_Gel_Velos_24_f15.mgf",
    "Fetal_Liver_Gel_Velos_24_f16.mgf",
    "Fetal_Liver_Gel_Velos_24_f17.mgf",
    "Fetal_Liver_Gel_Velos_24_f18.mgf",
    "Fetal_Liver_Gel_Velos_24_f19.mgf",
    "Fetal_Liver_Gel_Velos_24_f20.mgf",
    "Fetal_Liver_Gel_Velos_24_f21.mgf",
    "Fetal_Liver_Gel_Velos_24_f22.mgf",
    "Fetal_Liver_Gel_Velos_24_f23.mgf",
    "Fetal_Liver_Gel_Velos_24_f24.mgf",
    "Fetal_Liver_Gel_Velos_24_f25.mgf",
    "Fetal_Liver_Gel_Velos_24_f26.mgf",
    "Fetal_Liver_Gel_Velos_24_f27.mgf"
]

FETAL_TESTIS_BRP_ELITE_CRUX_FILES = [
    "Fetal_Testis_bRP_Elite_26_f01.mgf",
    "Fetal_Testis_bRP_Elite_26_f02.mgf",
    "Fetal_Testis_bRP_Elite_26_f03.mgf",
    "Fetal_Testis_bRP_Elite_26_f04.mgf",
    "Fetal_Testis_bRP_Elite_26_f05.mgf",
    "Fetal_Testis_bRP_Elite_26_f06.mgf",
    "Fetal_Testis_bRP_Elite_26_f07.mgf",
    "Fetal_Testis_bRP_Elite_26_f08.mgf",
    "Fetal_Testis_bRP_Elite_26_f09.mgf",
    "Fetal_Testis_bRP_Elite_26_f10.mgf",
    "Fetal_Testis_bRP_Elite_26_f11.mgf",
    "Fetal_Testis_bRP_Elite_26_f12.mgf",
    "Fetal_Testis_bRP_Elite_26_f13.mgf",
    "Fetal_Testis_bRP_Elite_26_f14.mgf",
    "Fetal_Testis_bRP_Elite_26_f15.mgf",
    "Fetal_Testis_bRP_Elite_26_f16.mgf",
    "Fetal_Testis_bRP_Elite_26_f17.mgf",
    "Fetal_Testis_bRP_Elite_26_f18.mgf",
    "Fetal_Testis_bRP_Elite_26_f19.mgf",
    "Fetal_Testis_bRP_Elite_26_f20.mgf",
    "Fetal_Testis_bRP_Elite_26_f21.mgf",
    "Fetal_Testis_bRP_Elite_26_f22.mgf",
    "Fetal_Testis_bRP_Elite_26_f23.mgf",
    "Fetal_Testis_bRP_Elite_26_f24.mgf",
    "Fetal_Testis_bRP_Elite_26_f25.mgf",
    "Fetal_Testis_bRP_Elite_26_f26.mgf",
    "Fetal_Testis_bRP_Elite_26_f27.mgf",
    "Fetal_Testis_bRP_Elite_26_f28.mgf",
    "Fetal_Testis_bRP_Elite_26_f29.mgf"
]

In [ ]:
IDENTIFICATIONS = [
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_Adrenalgland_bRP_Velos_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_Adrenalgland_Gel_Elite_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_Adrenalgland_Gel_Velos_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_Bcells_bRP_Elite_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_Bcells_bRP_Velos_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_Bcells_Gel_Elite_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_CD4Tcells_Gel_Velos_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_CD8Tcells_Gel_Elite_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_Colon_Gel_Elite_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_Esophagus_Gel_Velos_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_Heart_bRP_Velos_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_NKcells_bRP_Elite_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_NKcells_bRP_Velos_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_Pancreas_bRP_Elite_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_Platelets_Gel_Elite_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Adult_Urinarybladder_Gel_Elite_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Fetal_Brain_Gel_Velos_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Fetal_Gut_Gel_Velos_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Fetal_Liver_bRP_Elite_23_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Fetal_Liver_Gel_Velos_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Fetal_Ovary_bRP_Elite_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Fetal_Ovary_bRP_Velos_q_lt_0.01_identifications.tsv",
    "/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/mixedSpectraCrux/Fetal_Testis_bRP_Elite_q_lt_0.01_identifications.tsv"
]

MATCHES_TO_CRUX_FILES_LIST = {
    "Adult_Adrenalgland_Gel_Elite_q_lt_0.01_identifications.tsv" : ADULT_ADRENALGLAND_GEL_ELITE_CRUX_FILES,
    "Adult_Adrenalgland_Gel_Velos_q_lt_0.01_identifications.tsv" : ADULT_ADRENALGLAND_GEL_VELOS_CRUX_FILES,
    "Adult_Adrenalgland_bRP_Velos_q_lt_0.01_identifications.tsv" : ADULT_ADRENALGLAND_BRP_VELOS_CRUX_FILES,
    "Adult_Heart_bRP_Velos_q_lt_0.01_identifications.tsv" : ADULT_HEART_BRP_VELOS_CRUX_FILES,
    "Adult_Platelets_Gel_Elite_q_lt_0.01_identifications.tsv" : ADULT_PLATELETS_GEL_ELITE_CRUX_FILES,
    "Adult_Urinarybladder_Gel_Elite_q_lt_0.01_identifications.tsv" : ADULT_URINARYBLADDER_GEL_ELITE_CRUX_FILES,
    "Fetal_Brain_Gel_Velos_q_lt_0.01_identifications.tsv" : FETAL_BRAIN_GEL_VELOS_CRUX_FILES,
    "Fetal_Ovary_bRP_Velos_q_lt_0.01_identifications.tsv" : FETAL_OVARY_BRP_VELOS_CRUX_FILES,
    "Fetal_Ovary_bRP_Elite_q_lt_0.01_identifications.tsv" : FETAL_OVARY_BRP_ELITE_CRUX_FILES,

    "Adult_Bcells_bRP_Elite_q_lt_0.01_identifications.tsv" : ADULT_BCELLS_BRP_ELITE_CRUX_FILES,
    "Adult_Bcells_bRP_Velos_q_lt_0.01_identifications.tsv" : ADULT_BCELLS_BRP_VELOS_CRUX_FILES,
    "Adult_Bcells_Gel_Elite_q_lt_0.01_identifications.tsv" : ADULT_BCELLS_GEL_ELITE_CRUX_FILES,
    "Adult_CD4Tcells_Gel_Velos_q_lt_0.01_identifications.tsv" : ADULT_CD4TCELLS_GEL_VELOS_CRUX_FILES,
    "Adult_CD8Tcells_Gel_Elite_q_lt_0.01_identifications.tsv" : ADULT_CD8TCELLS_GEL_ELITE_CRUX_FILES,
    "Adult_Colon_Gel_Elite_q_lt_0.01_identifications.tsv" : ADULT_COLON_GEL_ELITE_CRUX_FILES,
    "Adult_Esophagus_Gel_Velos_q_lt_0.01_identifications.tsv" : ADULT_ESOPHAGUS_GEL_VELOS_CRUX_FILES,
    "Adult_NKcells_bRP_Elite_q_lt_0.01_identifications.tsv" : ADULT_NKCELLS_BRP_ELITE_CRUX_FILES,
    "Adult_NKcells_bRP_Velos_q_lt_0.01_identifications.tsv" : ADULT_NKCELLS_BRP_VELOS_CRUX_FILES,
    "Adult_Pancreas_bRP_Elite_q_lt_0.01_identifications.tsv" : ADULT_PANCREAS_BRP_ELITE_CRUX_FILES,
    "Fetal_Gut_Gel_Velos_q_lt_0.01_identifications.tsv" : FETAL_GUT_GEL_VELOS_CRUX_FILES,
    "Fetal_Liver_bRP_Elite_23_q_lt_0.01_identifications.tsv" : FETAL_LIVER_BRP_ELITE_23_CRUX_FILES,
    "Fetal_Liver_Gel_Velos_q_lt_0.01_identifications.tsv" : FETAL_LIVER_GEL_VELOS_CRUX_FILES,
    "Fetal_Testis_bRP_Elite_q_lt_0.01_identifications.tsv" : FETAL_TESTIS_BRP_ELITE_CRUX_FILES,

    "Adult_Adrenalgland_Gel_Elite_q_lt_0.001_identifications.tsv" : ADULT_ADRENALGLAND_GEL_ELITE_CRUX_FILES,
    "Adult_Adrenalgland_Gel_Velos_q_lt_0.001_identifications.tsv" : ADULT_ADRENALGLAND_GEL_VELOS_CRUX_FILES,
    "Adult_Adrenalgland_bRP_Velos_q_lt_0.001_identifications.tsv" : ADULT_ADRENALGLAND_BRP_VELOS_CRUX_FILES,
    "Adult_Heart_bRP_Velos_q_lt_0.001_identifications.tsv" : ADULT_HEART_BRP_VELOS_CRUX_FILES,
    "Adult_Platelets_Gel_Elite_q_lt_0.001_identifications.tsv" : ADULT_PLATELETS_GEL_ELITE_CRUX_FILES,
    "Adult_Urinarybladder_Gel_Elite_q_lt_0.001_identifications.tsv" : ADULT_URINARYBLADDER_GEL_ELITE_CRUX_FILES,
    "Fetal_Brain_Gel_Velos_q_lt_0.001_identifications.tsv" : FETAL_BRAIN_GEL_VELOS_CRUX_FILES,
    "Fetal_Ovary_bRP_Velos_q_lt_0.001_identifications.tsv" : FETAL_OVARY_BRP_VELOS_CRUX_FILES,
    "Fetal_Ovary_bRP_Elite_q_lt_0.001_identifications.tsv" : FETAL_OVARY_BRP_ELITE_CRUX_FILES,

    "Adult_Bcells_bRP_Elite_q_lt_0.001_identifications.tsv" : ADULT_BCELLS_BRP_ELITE_CRUX_FILES,
    "Adult_Bcells_bRP_Velos_q_lt_0.001_identifications.tsv" : ADULT_BCELLS_BRP_VELOS_CRUX_FILES,
    "Adult_Bcells_Gel_Elite_q_lt_0.001_identifications.tsv" : ADULT_BCELLS_GEL_ELITE_CRUX_FILES,
    "Adult_CD4Tcells_Gel_Velos_q_lt_0.001_identifications.tsv" : ADULT_CD4TCELLS_GEL_VELOS_CRUX_FILES,
    "Adult_CD8Tcells_Gel_Elite_q_lt_0.001_identifications.tsv" : ADULT_CD8TCELLS_GEL_ELITE_CRUX_FILES,
    "Adult_Colon_Gel_Elite_q_lt_0.001_identifications.tsv" : ADULT_COLON_GEL_ELITE_CRUX_FILES,
    "Adult_Esophagus_Gel_Velos_q_lt_0.001_identifications.tsv" : ADULT_ESOPHAGUS_GEL_VELOS_CRUX_FILES,
    "Adult_NKcells_bRP_Elite_q_lt_0.001_identifications.tsv" : ADULT_NKCELLS_BRP_ELITE_CRUX_FILES,
    "Adult_NKcells_bRP_Velos_q_lt_0.001_identifications.tsv" : ADULT_NKCELLS_BRP_VELOS_CRUX_FILES,
    "Adult_Pancreas_bRP_Elite_q_lt_0.001_identifications.tsv" : ADULT_PANCREAS_BRP_ELITE_CRUX_FILES,
    "Fetal_Gut_Gel_Velos_q_lt_0.001_identifications.tsv" : FETAL_GUT_GEL_VELOS_CRUX_FILES,
    "Fetal_Liver_bRP_Elite_23_q_lt_0.001_identifications.tsv" : FETAL_LIVER_BRP_ELITE_23_CRUX_FILES,
    "Fetal_Liver_Gel_Velos_q_lt_0.001_identifications.tsv" : FETAL_LIVER_GEL_VELOS_CRUX_FILES,
    "Fetal_Testis_bRP_Elite_q_lt_0.001_identifications.tsv" : FETAL_TESTIS_BRP_ELITE_CRUX_FILES
}

OUTPUT_FOLDER="/media/eduseiti/data_storage_2TB/unicamp/mixedSpectraCrux"

In [ ]:
STRUCT_FIELDS = "IId"
NO_PVALUE = sys.maxsize

In [ ]:
def decode_pvalues_file(pvalues_filename):
    
    pvalues = []

    with open(pvalues_filename, "rb") as inputFile:
        while True:
            record = inputFile.read(struct.calcsize(STRUCT_FIELDS))

            if not record:
                break
            else:
                unpacked = struct.unpack_from(STRUCT_FIELDS, record)
                
                pvalues.append(unpacked)
                
                if math.isnan(unpacked[2]):
                    print("nan: {}".format(record))

    print("Decoded {} pvalues from {}".format(len(pvalues), pvalues_filename))
    
    return np.array(pvalues)

In [ ]:
def plot_pvalues_histogram(pvalues):
    
    pvalues_df = pd.DataFrame(pvalues, columns = ["file", "scannr", "pvalue"])
    
    print(pvalues_df['pvalue'].describe(percentiles=list(np.round(np.arange(0.0, 1.0, 0.05), 2))))
    
    pvalues_histogram, pvalues_bin_edges = np.histogram(pvalues_df['pvalue'], 1000)

    fig = go.Figure()

    fig.add_trace(go.Bar(y=pvalues_histogram,
                         x=pvalues_bin_edges[1:],
                         marker_color='red'))
    
    fig.show()
    
    return pvalues_df, pvalues_histogram, pvalues_bin_edges

In [ ]:
pvalues = decode_pvalues_file(PVALUES_MIXEDSPECTRACRUX)

In [ ]:
pvalues_df, _, _ = plot_pvalues_histogram(pvalues)

In [ ]:
pvalues_df.columns

In [ ]:
pvalues_df = pvalues_df.sort_values(['file', 'scannr'])

In [ ]:
pvalues_df

In [ ]:
pvalues_idx = 0

current_experiment_first_general_file_idx = 0
current_experiment_total_files = 0

pvalues_array = pvalues_df.to_numpy()

FILE_IDX = pvalues_df.columns.get_loc("file")
SCANNR_IDX = pvalues_df.columns.get_loc("scannr")
PVALUE_IDX = pvalues_df.columns.get_loc("pvalue")

final_report = []

pvalues_identifications = []

for file in IDENTIFICATIONS:
        
    # Advance in pvalues file until finishing the previous experiment file scans
    
    while pvalues_array[pvalues_idx][FILE_IDX] < current_experiment_first_general_file_idx + current_experiment_total_files:
        pvalues_idx += 1

    # Update the general file index to start the current experiment
        
    current_experiment_first_general_file_idx += current_experiment_total_files
        
        
    # Read the current experiment data
        
    identifications_df = pd.read_csv(file, sep="\t")
    
    identifications_array = identifications_df.to_numpy()

    ID_FILE_IDX = identifications_df.columns.get_loc("file_idx")
    ID_SCAN = identifications_df.columns.get_loc("scan")
    
    
    filename = file.split('/')[-1]
        
    current_experiment_total_files = len(MATCHES_TO_CRUX_FILES_LIST[filename])  
    
    identifications_pvalues = []
    
    experiment_identifications_without_pvalue = 0
    processed_experiment_identifications = 0

    print("\n\nHandling experiment={}, with {} files and a with total of {} identifications".format(filename, 
                                                                                                  current_experiment_total_files, 
                                                                                                  identifications_df.shape[0]))
       

    last_processed_experiment_file = -1
    how_many_identifications_without_pvalue = 0
    
    total_identifications_experiment_file = 0
    identifications_without_pvalue_file = 0

    new_results = {}
        
    
    # Go through all the identification scans for the given file
        
    for identification in identifications_array:
        
        experiment_general_file_idx = current_experiment_first_general_file_idx + identification[ID_FILE_IDX]

        if last_processed_experiment_file != identification[ID_FILE_IDX]:
            
            if len(new_results) > 0:
                new_results['total identifications'] = total_identifications_experiment_file
                new_results['identifications with pvalue'] = total_identifications_experiment_file - identifications_without_pvalue_file
                new_results['identifications without pvalue'] = identifications_without_pvalue_file
                
                final_report.append(new_results)
                
                total_identifications_experiment_file = 0
                identifications_without_pvalue_file = 0                

                
            new_results = {}
            
            new_results['file'] = MATCHES_TO_CRUX_FILES_LIST[filename][identification[ID_FILE_IDX]]
            
            
            print("\n\n---- Filling pvalues for experiment file index={}, general file index={}. Starting pvalue_idx={}".format(identification[ID_FILE_IDX], 
                                                                                                                         experiment_general_file_idx, 
                                                                                                                         pvalues_idx))
            last_processed_experiment_file = identification[ID_FILE_IDX]
            
            
        processed_experiment_identifications += 1
        total_identifications_experiment_file += 1
        
        # Advance pvalues file until reaching the experiment general file index
        
        while pvalues_array[pvalues_idx][FILE_IDX] < experiment_general_file_idx:
            pvalues_idx += 1
        
#         print("-- Handling experiment scan={}, pvalues index={}".format(identification[ID_SCAN], pvalues_idx))

        found_identification_scan = False

        # Advance in pvalues file until reaching the identification scan

        while (pvalues_array[pvalues_idx][SCANNR_IDX] < identification[ID_SCAN]) and \
              (pvalues_array[pvalues_idx][FILE_IDX] == experiment_general_file_idx):
            pvalues_idx += 1

        if pvalues_array[pvalues_idx][FILE_IDX] == experiment_general_file_idx:
            if pvalues_array[pvalues_idx][SCANNR_IDX] == identification[ID_SCAN]:
                identifications_pvalues.append(pvalues_array[pvalues_idx][PVALUE_IDX])

#                 print("--- Found scan in pvalues index={}: {}".format(pvalues_idx, pvalues_array[pvalues_idx]))


                pvalues_identifications.append({"file": experiment_general_file_idx, 
                                                "scan": identification[ID_SCAN], 
                                                "pvalue": pvalues_array[pvalues_idx][PVALUE_IDX]})

                pvalues_idx += 1

                found_identification_scan = True


        if not found_identification_scan:
            identifications_pvalues.append(NO_PVALUE)

#             print("------ Identification scan={} had no pvalue".format(identification[ID_SCAN]))
            how_many_identifications_without_pvalue += 1
            identifications_without_pvalue_file += 1


    # Save report information of the last experiment file
    
    if len(new_results) > 0:
        new_results['total identifications'] = total_identifications_experiment_file
        new_results['identifications with pvalue'] = total_identifications_experiment_file - identifications_without_pvalue_file
        new_results['identifications without pvalue'] = identifications_without_pvalue_file

        final_report.append(new_results)    
    
    identifications_df['pvalue'] = identifications_pvalues

    identifications_df.to_csv(os.path.join(OUTPUT_FOLDER, file.split('/')[-1]), index=False, sep='\t')

    print("-- FINISHED after processing {}/{} identifications, with {} identifications scans without pvalues.".format(processed_experiment_identifications, 
                                                                                                                      identifications_df.shape[0],
                                                                                                                      how_many_identifications_without_pvalue))

final_report_df = pd.DataFrame(final_report)

datetime_execution = str(datetime.datetime.now())

final_report_df.to_csv(os.path.join(OUTPUT_FOLDER, "spectra_pvalues_report_" + datetime_execution + ".tsv"), index=False, sep='\t')
   
pvalues_identifications_df = pd.DataFrame(pvalues_identifications)
pvalues_identifications_df.to_csv(os.path.join(OUTPUT_FOLDER, "pvalues_identifications" + datetime_execution + ".tsv"), index=False, sep='\t')

In [ ]:
print(pvalues_identifications_df['pvalue'].describe(percentiles=list(np.round(np.arange(0.0, 1.0, 0.05), 2))))

pvalues_identifications_histogram, pvalues_identifications_bin_edges = np.histogram(pvalues_identifications_df['pvalue'], 1000)

fig = go.Figure()

fig.add_trace(go.Bar(y=pvalues_identifications_histogram,
                     x=pvalues_identifications_bin_edges[1:],
                     marker_color='red'))

fig.show()

In [ ]:
pvalues_df['pvalue'].describe(percentiles=list([0.1, 0.01, 0.001]))

In [ ]:
pvalues_identifications_df['pvalue'].describe(percentiles=list([0.1, 0.01, 0.001]))

### Create a report with the statistics for each individual file

In [ ]:
count_df = pd.read_csv(COUNTS_MIXEDSPECTRACRUX, sep='\t')

In [ ]:
final_report_df

In [ ]:
final_report_totals_df = count_df.set_index('file').join(final_report_df.set_index('file'))

In [ ]:
final_report_totals_df.to_csv(os.path.join(OUTPUT_FOLDER, "final_report_with_totals" + datetime_execution + ".tsv"), index=False, sep='\t')

In [ ]:
final_report_totals_df.sum()